# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

D:\software\anaconda\envs\notebook\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据

In [2]:
import pandas as pd

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data = data.dropna() # 删除空值？？？
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## Step3 创建DATASET

In [4]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        
        super().__init__()
        
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        
        return len(self.data)

In [5]:
dataset = MyDataset()

for i in range(5):

    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [6]:
from torch.utils.data import random_split

trainset, validset = random_split(dataset, lengths=[0.9, 0.1])

len(trainset), len(validset)

(6989, 776)

In [7]:
for i in range(10):
    
    print(trainset[i])

('1、总体感觉有些旧，地毯上污迹比较多。2、服务等还可以。3、房间总带有一些烟味，不喜欢。', 1)
('春节期间入住，性价比高，还不错，但是不知道平时多少钱，在成都知名度高，住着比较有面子，但是房间太小了，而且显得旧，可能和当初的设计有关系，门口都是单行道不是很方便，如果价钱还是这么可人的话，下次还会选择。', 1)
('很不错的司星酒店，已经是为我们的贵宾客户第四次预订了，预定的行政豪华房，非常不错，只是房间略显小了些，但总体很好，服务业很好！值得推荐！', 1)
('距离旺角、尖沙咀等商业区非常遥远，就算是去海洋公园也很远。房价虽然在酒店里面算是比较便宜的一个档次，但是如果你要去购物逛街，所需要的车费也不便宜，要坐巴士还要转地铁，浪费很多时间。算是一个比较偏僻的地方。虽然房间环境不错，但是下次再去香港也不敢选择这个酒店，因为交通很不方便。而且酒店附近很多店都很早就关门了，晚上10点左右想买东西吃的话，都不知道可以去哪里买，只能去附近唯一一间比较晚关门的惠康超市里面买泡面吃了。', 1)
('总体评价算2星就不错了，性价比低，是我有史以来住宿最差的了，比东南亚的一些小国酒店还差，不过没办法，遂宁只有2家4星的，另一家明星康年（据说好些，且价格低）当时无客房，只有住宿天友了。1.床太小了，尤其情侣住宿，不舒服，晚上转个姿势，差点落下去2.室内洗漱用品太小气了，只有2包沐浴露，2包飘柔（便利店出售的那种小包）3.上网收费很不符合星级酒店惯例4.室内装饰几乎没有，连一般酒店介绍本地风土人情的宣传资料都没有5.早餐种类很少，而且供应的很少，一会什么东西都没的吃了，我们只吃了一次，其他时间自己出去吃了', 1)
('海滩有点名过其实了，也确实远了一点，交通也不方便，但酒店本身是很不错的。在尖沙咀半岛酒店（靠近香港青年会的那一边）有黄金海岸酒店自己的专线巴士，来回都是免费的（楼上有说要收费的，可能是和司机沟通不好）。班次可以到酒店的网站下载（，不建议其它交通方式，这是我的忠告！附近有超市，茶餐厅和麦当劳，要求不高的话吃是没问题的。', 1)
('地点一流，房间设施比较陈旧，价格也越来越高了。好象携程的优势越来越不明显了。', 1)
('电话预定春节初一到初四入住，标准房338元，含双早。初一晚11点到宾馆，因房间满免费升级至商务房。房间干净。离地铁站近。仅打了一次车，从西

## Step5 创建DATALOADER

In [8]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    
    texts, labels = [], []
    
    for item in batch:
    
        texts.append(item[0])
        
        labels.append(item[1])
    
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    
    inputs["labels"] = torch.tensor(labels)
    
    return inputs

In [9]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [10]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 6821, 3613,  ..., 1218,  782,  102],
        ...,
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 3301, 1351,  ...,    0,    0,    0],
        [ 101, 6421, 2421,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 0, 0

## Step6 创建模型及优化器

In [11]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    #
    model = model.cuda()

Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3

In [12]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [13]:
def evaluate():
    
    model.eval()
    
    acc_num = 0
    
    with torch.inference_mode():
    
        for batch in validloader:
        
            if torch.cuda.is_available():
            
                batch = {k: v.cuda() for k, v in batch.items()}
            
            output = model(**batch)
            
            pred = torch.argmax(output.logits, dim=-1)
            
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    
    global_step = 0
    
    for ep in range(epoch): # 分EPOCH
    
        model.train() # 设置TRAIN模式？？？？
        
        for batch in trainloader: # 得到一个BATCH
        
            if torch.cuda.is_available():
            
                batch = {k: v.cuda() for k, v in batch.items()} # 将VALUE转成CUDA
            
            optimizer.zero_grad() # 优化器复位
            
            output = model(**batch)
            
            output.loss.backward()
            
            optimizer.step()
            
            if global_step % log_step == 0:
            
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            
            global_step += 1
        
        acc = evaluate() # 评估
        
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [21]:
train(epoch=10)

ep: 0, global_step: 0, loss: 0.056225765496492386
ep: 0, global_step: 100, loss: 0.28883278369903564
ep: 0, global_step: 200, loss: 0.37623557448387146
ep: 0, acc: 0.8865979313850403
ep: 1, global_step: 300, loss: 0.2853756546974182
ep: 1, global_step: 400, loss: 0.0687469094991684
ep: 1, acc: 0.894329845905304
ep: 2, global_step: 500, loss: 0.028002846986055374
ep: 2, global_step: 600, loss: 0.0042667449451982975
ep: 2, acc: 0.8865979313850403
ep: 3, global_step: 700, loss: 0.0333065465092659
ep: 3, global_step: 800, loss: 0.05123006924986839
ep: 3, acc: 0.8775773048400879
ep: 4, global_step: 900, loss: 0.003583570709452033
ep: 4, global_step: 1000, loss: 0.006680651102215052
ep: 4, acc: 0.8917525410652161
ep: 5, global_step: 1100, loss: 0.012078236788511276
ep: 5, global_step: 1200, loss: 0.08373112231492996
ep: 5, global_step: 1300, loss: 0.2946608066558838
ep: 5, acc: 0.8801546096801758
ep: 6, global_step: 1400, loss: 0.0015973811969161034
ep: 6, global_step: 1500, loss: 0.00286248

## Step9 模型预测

In [18]:
sen = "我觉得这家酒店不错，饭很好吃！"

id2_label = {0: "差评！", 1: "好评！"}

model.eval()

with torch.inference_mode():

    inputs = tokenizer(sen, return_tensors="pt")
    
    inputs = {k: v.cuda() for k, v in inputs.items()}
    
    logits = model(**inputs).logits
    
    pred = torch.argmax(logits, dim=-1)
    
    print(f"输入：{sen}\n模型预测结果：{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果：好评！


In [19]:
from transformers import pipeline

model.config.id2label = id2_label

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [20]:
pipe(sen)

[{'label': '好评！', 'score': 0.9927247762680054}]